## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

In [2]:
import random
import pandas as pd
import string
import time


## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [2]:
N = 1000000
A = [random.randint(0, 1000) for _ in range(N)]
B = [a + 100 for a in A]
srzB = sum(B) / N 
print(srzB)

599.928091


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [9]:
# создаем таблицу
n_rows = 2000000
df = pd.DataFrame({'Цифры1': [random.random() for _ in range(n_rows)],
                   'Цифры2': [random.random() for _ in range(n_rows)],
                   'Цифры3': [random.random() for _ in range(n_rows)],
                   'Цифры4': [random.random() for _ in range(n_rows)]})

# добавляем столбец key со случайными английскими буквами
def random_letter():
    return random.choice(string.ascii_lowercase)

df['Буквы'] = [''.join([random_letter() for _ in range(5)]) for _ in range(n_rows)]

# выбираем подмножество строк с первыми 5 буквами в столбце key
subset = df[df['Буквы'].str[:5] == 'abcde']

print(df)
print(subset)

           Цифры1    Цифры2    Цифры3    Цифры4  Буквы
0        0.872280  0.612722  0.868016  0.634420  ozluv
1        0.331172  0.461838  0.997744  0.368127  ubufp
2        0.792951  0.122124  0.017367  0.599376  xbiev
3        0.939772  0.712012  0.179982  0.096605  dczfm
4        0.051715  0.747909  0.130269  0.827119  ctogw
...           ...       ...       ...       ...    ...
1999995  0.159621  0.708859  0.737085  0.662575  afxzr
1999996  0.871066  0.180411  0.336052  0.077756  vdaki
1999997  0.593704  0.000739  0.427464  0.024195  eicno
1999998  0.903016  0.449299  0.499921  0.835120  nhsxd
1999999  0.357188  0.723243  0.774772  0.850451  hlhnv

[2000000 rows x 5 columns]
Empty DataFrame
Columns: [Цифры1, Цифры2, Цифры3, Цифры4, Буквы]
Index: []


## Лабораторная работа 3

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [9]:
# загрузка данных
recipes = pd.read_csv('recipes_sample.csv', index_col=0)
reviews = pd.read_csv('reviews_sample.csv', index_col=0)

# приводим столбцы к нужному типу
reviews['date'] = pd.to_datetime(reviews['date'])
reviews['rating'] = reviews['rating'].astype(float)

print(reviews)


# функция подсчета среднего значения с использованием метода DataFrame.iterrows исходной таблицы
def mean_rating_iterrows(reviews):
    total = 0
    count = 0
    for index, row in reviews.iterrows():
        if row['date'].year == 2010:
            total += row['rating']
            count += 1
        if count > 0:
            return total/count
        else:
            return 0
# функция подсчета среднего значения с использованием метода DataFrame.iterrows таблицы, в которой сохранены только отзывы за 2010 год
def mean_rating_iterrows_2010(reviews):
    total = 0
    count = 0
    for index, row in reviews[reviews['date'].dt.year == 2010].iterrows():
        total += row['rating']
        count += 1
        if count > 0:
            return total/count
        else:
            return 0
        
# функция подсчета среднего значения с использованием метода Series.mean
def mean_rating_mean(reviews):
    return reviews[reviews['date'].dt.year == 2010]['rating'].mean()

# проверка корректности работы функций и измерение времени выполнения
start_time = time.time()
result_1 = mean_rating_iterrows(reviews)
print(f"Результат функции mean_rating_iterrows: {result_1}, время выполнения: {time.time() - start_time} сек")

start_time = time.time()
result_2 = mean_rating_iterrows_2010(reviews)
print(f"Результат функции mean_rating_iterrows_2010: {result_2}, время выполнения: {time.time() - start_time} сек")

start_time = time.time()
result_3 = mean_rating_mean(reviews)
print(f"Результат функции mean_rating_mean: {result_3}, время выполнения: {time.time() - start_time} сек")

assert round(result_1, 2) == 2.92
assert round(result_2, 2) == 2.92
assert round(result_3, 2) == 2.92 

            user_id  recipe_id       date  rating  \
370476        21752      57993 2003-05-01     5.0   
624300       431813     142201 2007-09-16     5.0   
187037       400708     252013 2008-01-10     4.0   
706134   2001852463     404716 2017-12-11     5.0   
312179        95810     129396 2008-03-14     5.0   
...             ...        ...        ...     ...   
1013457     1270706     335534 2009-05-17     4.0   
158736      2282344       8701 2012-06-03     0.0   
1059834      689540     222001 2008-04-08     5.0   
453285   2000242659     354979 2015-06-02     5.0   
691207       463435     415599 2010-09-30     5.0   

                                                    review  
370476   Last week whole sides of frozen salmon fillet ...  
624300   So simple and so tasty!  I used a yellow capsi...  
187037   Very nice breakfast HH, easy to make and yummy...  
706134   These are a favorite for the holidays and so e...  
312179   Excellent soup!  The tomato flavor is just gre...

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [10]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


#### [версия 2]
* Уточнены формулировки задач 1, 3, 4